In [130]:
import pandas as pd
import uuid
import random
from datetime import datetime, timedelta
import sqlite3
import os

In [146]:
output_dir = r"C:\Users\darshan.bapodariya\Downloads\onsite task"

# Ensure directory exists
os.makedirs(output_dir, exist_ok=True)
campaigns_df = pd.read_csv(os.path.join(output_dir, "campaigns.csv"), index_col=False)
impressions_df = pd.read_csv(os.path.join(output_dir, "impressions.csv"), index_col=False)
conversions_df = pd.read_csv(os.path.join(output_dir, "conversions.csv"), index_col=False)
impressions_df

,impression_id,user_id,campaign_id,impression_ts
0,3130f6db-4e74-4eaf-afb0-6a1aecbd9765,49,CAMP101,15/03/2025 04:12
1,e860dc10-f741-4be4-9cb9-bc9d1c8e8f62,40,CAMP100,02/03/2025 03:54
2,b823a37a-17bd-4d0d-a369-72e68e70f81f,30,CAMP109,31/03/2025 14:30
3,68f87f51-f11b-4acb-a71e-8c2cb890d42a,41,CAMP101,08/03/2025 00:10
4,863534ab-cc7d-4759-8bd7-7ae9588b2502,27,CAMP106,07/04/2025 01:09
...,...,...,...,...
513,05b8e56b-7403-4eb0-a425-e1b5b19de162,67,CAMP108,06/04/2025 10:59
514,6bd0d4fc-02ca-48c8-aee3-13b85909c5bb,82,CAMP104,29/03/2025 00:49
515,ef091181-1a99-437f-9a54-0ed2f8bb18b5,29,CAMP104,27/03/2025 10:17
516,36a82ab6-1e47-4908-8556-a4a8c2125923,84,CAMP106,01/04/2025 09:37


# Task 1: Data cleaning

### 1. Identifies duplicates : 

### 2. Validates foreign key relationships :

### 3. Reports issues :

Return two DataFrames (or result sets):

duplicate_records: showing duplicates with count.

invalid_conversions: showing conversions with campaign_ids not found in campaigns.

In [144]:
# 1. Identify duplicate impression_ids
dup_impressions = impressions_df.groupby("impression_id").size().reset_index(name='count')
duplicate_impressions = dup_impressions[dup_impressions['count'] > 1]

# 2. Identify duplicate conversion_ids
dup_conversions = conversions_df.groupby("conversion_id").size().reset_index(name='count')
duplicate_conversions = dup_conversions[dup_conversions['count'] > 1]

# Combine duplicates
duplicate_records = pd.concat([
    duplicate_impressions.assign(type='impression'),
    duplicate_conversions.assign(type='conversion')
])

# 3. Validate foreign keys: campaign_id in conversions must exist in campaigns
valid_campaign_ids = set(campaigns_df['campaign_id'])
invalid_conversions = conversions_df[~conversions_df['campaign_id'].isin(valid_campaign_ids)]

# Outputs
print("Duplicate Records:")
print(duplicate_records)

print("\nInvalid Conversions (invalid campaign_id):")
print(invalid_conversions)


Duplicate Records:
                            impression_id  count        type  \
7    05b8e56b-7403-4eb0-a425-e1b5b19de162      2  impression   
15   0bd2dc3d-6e17-41a1-97d3-78d9111eed10      2  impression   
26   13010dd0-2694-40ad-9c29-173a003f8f2b      2  impression   
34   1809af8f-4dc6-4507-ab17-55197dee39e6      2  impression   
35   18c75ccd-8a07-41af-a964-8d1cc62cf288      2  impression   
37   19868776-6c1e-4a23-8a6d-8d22ceeeec0b      2  impression   
51   209e1a9d-e512-4a00-a238-30014163d2e4      2  impression   
52   20c0d80a-6222-4262-b9ce-ff357aea8217      2  impression   
62   25342125-04f8-40ea-8fd6-f4b826a45f53      2  impression   
68   28c368ad-4019-434d-b977-aa268a25223e      2  impression   
77   2ebc8d10-e58e-4447-99d7-64910b17464c      2  impression   
83   3162e1a8-22ed-4659-80d6-c43a34744543      2  impression   
87   32d65dd2-0c6b-44f8-8afc-cd14b8dc801b      2  impression   
93   36a82ab6-1e47-4908-8556-a4a8c2125923      2  impression   
103  3e1d1120-da8d-4d

In [159]:
# Step 1: Remove duplicates from impressions_df
# Keep the first occurrence of each impression_id
impressions_df = impressions_df.drop_duplicates(subset="impression_id", keep="first")

# Step 2: Remove duplicates from conversions_df
# Keep the first occurrence of each conversion_id
conversions_df = conversions_df.drop_duplicates(subset="conversion_id", keep="first")

# Merge conversions with impressions
merged = pd.merge(conversions_df, impressions_df, on=["user_id", "campaign_id"], suffixes=("_conv", "_imp"))

In [161]:
# Convert type to timestamps
merged["conversion_ts"] = pd.to_datetime(merged["conversion_ts"])
merged["impression_ts"] = pd.to_datetime(merged["impression_ts"])


C:\Users\darshan.bapodariya\AppData\Local\Temp\ipykernel_16464\3885612996.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged["conversion_ts"] = pd.to_datetime(merged["conversion_ts"])
C:\Users\darshan.bapodariya\AppData\Local\Temp\ipykernel_16464\3885612996.py:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged["impression_ts"] = pd.to_datetime(merged["impression_ts"])


# Task 2: filterout conversion withing 30 mins
## Conversion must be done in campagin duration. 

In [168]:
# Filter: conversion within 30 min of impression
merged["time_diff"] = (merged["conversion_ts"] - merged["impression_ts"]).dt.total_seconds() / 60.0
merged_30 = merged[(merged["time_diff"] >= 0) & (merged["time_diff"] <= 30)]

# Join with campaign metadata
merged_30 = merged_30.merge(campaigns_df, on="campaign_id")
merged_30 = merged_30[
    (merged_30["conversion_ts"] >= merged_30["start_date"]) &
    (merged_30["conversion_ts"] <= merged_30["end_date"])
]

# Final output
final_df = merged_30[[
    "conversion_id",
    "user_id",
    "campaign_name",
    "channel",
    "conversion_value",
    "conversion_ts",
    "time_diff"
]]



In [170]:
final_df

,conversion_id,user_id,campaign_name,channel,conversion_value,conversion_ts,time_diff
0,e3f3085a-9da8-41d3-bb1f-8dca4bc925b4,97,Campaign_7,Social,209.46,2025-03-22 12:26:00,17.0
1,569267d1-8892-4a5d-a488-44c5342b60d5,31,Campaign_4,Email,235.47,2025-03-24 20:54:00,15.0
2,16dbf663-5424-4b86-a8d8-2fd198209d52,48,Campaign_8,Social,371.93,2025-04-11 17:22:00,5.0
3,7d6d3660-d451-4526-a759-4a4582bcd5ea,18,Campaign_3,Social,494.39,2025-03-26 16:14:00,12.0
4,d946de16-ca45-4c5c-a8a7-cc2b4d0d76a1,15,Campaign_2,Display,101.57,2025-03-14 00:12:00,11.0
...,...,...,...,...,...,...,...
94,f256e5c5-4466-4912-8d4d-a0526ab3e2e1,39,Campaign_9,OOH,27.51,2025-04-01 08:02:00,15.0
95,a6881a5d-89ef-44f1-92ad-c847ca0f3e30,71,Campaign_5,OOH,184.24,2025-03-30 22:08:00,18.0
96,1aa72823-984c-474e-8d6a-0b68f33ce1b5,46,Campaign_4,Email,13.50,2025-03-13 13:03:00,25.0
97,c15ff915-c49d-47f0-a967-ce7f10a71104,45,Campaign_5,OOH,411.05,2025-03-20 01:41:00,24.0


# Task 2

## A conversion is only valid if:

### The conversion_ts is within 45 minutes of the impression_ts.

### Only the first valid conversion per user per campaign should be counted.

### Average time to convert per campagins should be in minutes (rounded to 2 decimal places).

In [172]:
# Filter: conversion is after impression and within 45 minutes
valid = merged[
    (merged["conversion_ts"] > merged["impression_ts"]) &
    (merged["conversion_ts"] <= merged["impression_ts"] + timedelta(minutes=45))
]

# Sort and deduplicate: keep only the first conversion per user per campaign
valid = valid.sort_values(by="conversion_ts")
valid = valid.drop_duplicates(subset=["user_id", "campaign_id"], keep="first")


# Compute time to convert in minutes
valid["time_to_convert"] = (valid["conversion_ts"] - valid["impression_ts"]).dt.total_seconds() / 60

In [174]:

# Aggregation
summary = valid.groupby("campaign_id").agg(
    total_conversions=('conversion_id', 'count'),
    total_conversion_value=('conversion_value', 'sum'),
    avg_time_to_convert_mins=('time_to_convert', 'mean')
).reset_index()

# Total impressions per campaign
impression_counts = impressions_df.groupby("campaign_id").agg(
    total_impressions=('impression_id', 'count')
).reset_index()

# Final result
result = pd.merge(impression_counts, summary, on="campaign_id", how="left")
result = result.fillna({
    "total_conversions": 0,
    "total_conversion_value": 0.0,
    "avg_time_to_convert_mins": 0.0
})

# Round the average time
result["avg_time_to_convert_mins"] = result["avg_time_to_convert_mins"].round(2)

print(result)


  campaign_id  total_impressions  total_conversions  total_conversion_value  \
0     CAMP100                 42                 13                 3465.91   
1     CAMP101                 47                 14                 3497.30   
2     CAMP102                 51                 18                 4678.57   
3     CAMP103                 42                 12                 3556.38   
4     CAMP104                 51                 11                 2575.49   
5     CAMP105                 54                 12                 3328.27   
6     CAMP106                 55                 15                 3491.91   
7     CAMP107                 45                 13                 3219.10   
8     CAMP108                 50                 15                 4387.92   
9     CAMP109                 49                 14                 3688.01   

   avg_time_to_convert_mins  
0                     26.00  
1                     20.00  
2                     17.61  
3         